Loading Libraries

In [45]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense 
import nltk 
import re 
from nltk.corpus import stopwords 
### Dataset Preprocessing 
from nltk.stem.porter import PorterStemmer ##stemming purpose 



Loading dataset

In [46]:
df=pd.read_csv('/content/drive/MyDrive/NLP/Fake_News Classifier/LSTM_Stemm/LSTM_Lemmnatization/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [47]:
df=df.dropna()

In [48]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [49]:
df.shape

(18285, 5)

In [50]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

Get independent Featyures

In [51]:

X=df.drop('label',axis=1)

Dependent Features

In [52]:
y=df['label']

In [8]:
tf.__version__

'2.8.2'

Vocabulary Size

In [53]:
voc_size=5000

In [54]:
message=X.copy()

In [55]:
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [56]:
message.reset_index(inplace=True)

In [57]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
ps = PorterStemmer()
lm = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

corpus=[]
for i in range(0,len(message)):
  review = re.sub('[^a-zA-Z]', ' ',message['title'][i])
  review = review.lower()
  review = review.split()

  
  review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english') ] # lemma
  review = ' '.join(review)
  corpus.append(review)




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [59]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted'

In [60]:
onehot_rep=[one_hot(word,voc_size) for word in corpus]
onehot_rep[0]

[601, 506, 1048, 3141, 3365, 484, 4647, 4853, 3384, 36]

Padding

In [62]:
#Padding
sent_length= 20
embedded_docs = pad_sequences(onehot_rep,padding='post',maxlen=sent_length)
embedded_docs

array([[ 601,  506, 1048, ...,    0,    0,    0],
       [ 244,  748, 4993, ...,    0,    0,    0],
       [4913, 2505, 1167, ...,    0,    0,    0],
       ...,
       [2659, 4219, 1009, ...,    0,    0,    0],
       [1561, 2363, 1460, ...,    0,    0,    0],
       [3630,  458, 1631, ...,    0,    0,    0]], dtype=int32)

In [63]:
embedded_docs[0]

array([ 601,  506, 1048, 3141, 3365,  484, 4647, 4853, 3384,   36,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

Model Creation

In [64]:
embedding_vector_features=100
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 100)           500000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embedded_docs,y,test_size=0.33,random_state=42)

In [66]:
# final training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 9s 10ms/step - loss: 0.2984 - accuracy: 0.8593 - val_loss: 0.1965 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 1s 8ms/step - loss: 0.1249 - accuracy: 0.9526 - val_loss: 0.2088 - val_accuracy: 0.9173
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0748 - accuracy: 0.9736 - val_loss: 0.2384 - val_accuracy: 0.9122
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0451 - accuracy: 0.9852 - val_loss: 0.3353 - val_accuracy: 0.9031
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0321 - accuracy: 0.9893 - val_loss: 0.3639 - val_accuracy: 0.9104
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0223 - accuracy: 0.9944 - val_loss: 0.3812 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0151 - accuracy: 0.9962 - val_loss: 0.4015 - val_accuracy: 0.9140
Epoch

In [69]:
# Accuracy check
import numpy as np
y_pred = model.predict(X_test)
y_pred= np.where(y_pred>0.6,1,0) # AUC ROC curve             

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9032311516155758

In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      3419
           1       0.88      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

